In [12]:
from kazoo.client import KazooClient
from threading import Thread
from time import sleep

In [13]:
# Create the root node ZOO
kc = KazooClient()
kc.start()
if kc.exists('/zoo'):
    kc.delete('/zoo', recursive=True)
kc.create('/zoo')

N_ANIMALS = 2

Current children list: ['a2']
Current data list: None
<class 'NoneType'>
Current children list: []
Current children list: []
Current data list: None
<class 'NoneType'>
Current children list: []
Current data list: b'' ZnodeStat(czxid=19247, mzxid=19247, ctime=1640018142684, mtime=1640018142684, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=0, numChildren=0, pzxid=19247) WatchedEvent(type='DELETED', state='CONNECTED', path='/zoo')
Current data list: b'' ZnodeStat(czxid=19247, mzxid=19247, ctime=1640018142684, mtime=1640018142684, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=0, numChildren=0, pzxid=19247) None
Current children list: []


In [14]:
barrier = kc.Barrier('/zoo')

In [15]:
barrier.create()

In [21]:
barrier.remove()

False

Current data list: b'' Current data list:ZnodeStat(czxid=19378, mzxid=19378, ctime=1640018598267, mtime=1640018598267, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=0, numChildren=0, pzxid=19378) WatchedEvent(type='CREATED', state='CONNECTED', path='/zoo')
 b'' ZnodeStat(czxid=19378, mzxid=19378, ctime=1640018598267, mtime=1640018598267, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=0, numChildren=0, pzxid=19378) WatchedEvent(type='CREATED', state='CONNECTED', path='/zoo')
Current data list:Current data list: None None WatchedEvent(type='DELETED', state='CONNECTED', path='/zoo')
 None None WatchedEvent(type='DELETED', state='CONNECTED', path='/zoo')


In [17]:
barrier.wait()

True

Current data list:Current data list: None None WatchedEvent(type='DELETED', state='CONNECTED', path='/zoo')
 None None WatchedEvent(type='DELETED', state='CONNECTED', path='/zoo')


In [6]:
# Create a watch for children of the root

@kc.ChildrenWatch('/zoo')
def watch_children(children):
    print('Current children list:', children)
    if len(children) == N_ANIMALS:
        print('Zoo is full.')

Current children list: []


In [8]:
# Our children `process`
class Animal(Thread):
    def __init__(self, root: str, name: str, run_delay: int = 3):
        super().__init__()   
        self.kc = KazooClient()
        self.kc.start()
        self.url = f'{root}/{name}'
        self.run_delay = run_delay
        @kc.DataWatch(self.url)
        def watch_data(data, stat):
            print('Current data list:', data)
            print(type(data))
            return True

    def enter(self):
        self.kc.create(self.url, ephemeral=True)
    
    def run(self):
        print(f'Animal with url={self.url} started running')
        print('Animal is screaming!')
        self.kc.set(self.url, self.url.encode())
        sleep(self.run_delay)
        self.kc.stop()
        print(f'Animal with url={self.url} stopped running')

In [9]:
a1 = Animal('zoo', 'a1')
a2 = Animal('zoo', 'a2')

Current data list: None
<class 'NoneType'>
Current data list: None
<class 'NoneType'>


In [10]:
a1.enter()

Current children list: ['a1']
Current data list: b''
<class 'bytes'>
Current children list: ['a1']


In [11]:
a2.enter()

Current children list: ['a1', 'a2']
Zoo is full.
Current data list: b''
<class 'bytes'>
Current children list: ['a1', 'a2']
Zoo is full.


In [15]:
# We see that zoo is full, we can release our animals
a1.start()

Animal with url=zoo/a1 started running
Animal is screaming!
Current data list: b'zoo/a1' ZnodeStat(czxid=209, mzxid=211, ctime=1640013557744, mtime=1640013565596, version=1, cversion=0, aversion=0, ephemeralOwner=72057598961844276, dataLength=6, numChildren=0, pzxid=209) WatchedEvent(type='CHANGED', state='CONNECTED', path='/zoo/a1')
Animal with url=zoo/a1 stopped running
Current data list: None None WatchedEvent(type='DELETED', state='CONNECTED', path='/zoo/a1')
Current children list: ['a2']


In [10]:
a2.start()

Animal with url=zoo/a2 started running
Animal is screaming!
Animal with url=zoo/a1 stopped running
Current children list: ['a2']
Animal with url=zoo/a2 stopped running
Current children list: []


In [11]:
kc.delete('/zoo', recursive=True)

Animal with url=zoo/a1 stopped running
Current children list: ['a2']
Current children list: []
Current data list: None None WatchedEvent(type='DELETED', state='CONNECTED', path='/zoo')
Animal with url=zoo/a2 stopped running
